# import and read

In [294]:
import pandas as pd
import os
import sqlalchemy as sql
import numpy as np
import sqlite3

In [295]:
# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\Zensus\\input\\Zensus.xlsx'
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)
data_frames = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')}

# check
print("Dateninfrastruktur aufgesetzt und Excel-Daten geladen.")

Dateninfrastruktur aufgesetzt und Excel-Daten geladen.


In [296]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = {}
for sheet_name, data in data_frames.items():
    # Ersetzen von '-' und '/' durch '_' in den Tabellenblätternamen
    new_sheet_name = sheet_name.replace('-', '_').replace('/', '_')
    
    # Ersetzen von '.', '/', '-' und '–' durch 0 in den Daten
    df = data.apply(lambda x: x.map(lambda y: 0 if y in ['.', '/', '-', '–'] else y))
    
    # Bereinigung der 'Regionalebene'-Spalte (Kreise)
    if 'Regionalebene' in df.columns:
        df['Regionalebene'] = df['Regionalebene'].str.replace('/', '_')

    # Umwandlung in numerische Typen für Spalten ab der 5.
    for col in df.columns[4:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    globals()[new_sheet_name] = df
    created_dfs[new_sheet_name] = df

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs.keys():
    print(df_name)

Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


# EDA

In [297]:
CSV_Klassenstufe.groupby('Regionalebene')['SCHUELER_KLSS_STP'].sum()

Regionalebene
Bund                                     8903780
Gemeinde                                 6631640
Gemeindeverband                          1310750
Land                                     8903780
Regierungsbezirk                         7013880
Stadtkreis_kreisfreie Stadt_Landkreis    8903770
Name: SCHUELER_KLSS_STP, dtype: int64

# transform

## regionalebenen

In [298]:
# Definieren der Regionalebenen
liste = [
    'Bund',
    'Gemeinde',
    'Gemeindeverband',
    'Land',
    'Regierungsbezirk',
    'Stadtkreis_kreisfreie_Stadt_Landkreis'
]

In [299]:

# Erstellen der Tabellen für alle Regionalebenen
regionalebenen = {}
for ebene in liste:
    regionalebenen[ebene] = pd.DataFrame()
    for df in created_dfs.values():
        if 'Regionalebene' in df.columns:
            filtered_df = df[df['Regionalebene'] == ebene][['_RS', 'Name']]
            regionalebenen[ebene] = pd.concat([regionalebenen[ebene], filtered_df], ignore_index=True)
    regionalebenen[ebene] = regionalebenen[ebene].drop_duplicates().reset_index(drop=True)

    # check
    print(f"Tabelle für {ebene} wurde erfolgreich erstellt.")
    print(regionalebenen[ebene].head())

Tabelle für Bund wurde erfolgreich erstellt.
   _RS         Name
0    0  Deutschland
Tabelle für Gemeinde wurde erfolgreich erstellt.
           _RS                    Name
0  10010000000        Flensburg, Stadt
1  10020000000  Kiel, Landeshauptstadt
2  10030000000      Lübeck, Hansestadt
3  10040000000       Neumünster, Stadt
4  10510011011      Brunsbüttel, Stadt
Tabelle für Gemeindeverband wurde erfolgreich erstellt.
        _RS                    Name
0  10515163  Burg-St. Michaelisdonn
1  10515166           Marne-Nordsee
2  10515169                   Eider
3  10515172           Heider Umland
4  10515175      Mitteldithmarschen
Tabelle für Land wurde erfolgreich erstellt.
   _RS                 Name
0    1   Schleswig-Holstein
1    2              Hamburg
2    3        Niedersachsen
3    4               Bremen
4    5  Nordrhein-Westfalen
Tabelle für Regierungsbezirk wurde erfolgreich erstellt.
   _RS                              Name
0   31  Statistische Region Braunschweig
1   32  

In [300]:
# Erstellen der Spalten für Gemeindeverbandsschlüssel (erste 8 Stellen)
for ebene in ['Gemeinde', 'Gemeindeverband']:
    regionalebenen[ebene]['Gemeindeverbandsschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:8].astype(int)

# Erstellen der Spalten für Kreisschlüssel (erste 4 Stellen)
for ebene in ['Gemeindeverband', 'Stadtkreis_kreisfreie_Stadt_Landkreis']:
    regionalebenen[ebene]['Kreisschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:4].astype(int)

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['Stadtkreis_kreisfreie_Stadt_Landkreis', 'Land']:
    regionalebenen[ebene]['Landesschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:2].astype(int)

# Alle Tabellen ohne die Spalte '_RS' speichern (ausser Gemeinde)
for ebene, df in regionalebenen.items():
    if ebene != 'Gemeinde':
        regionalebenen[ebene] = df.drop('_RS', axis=1)

# _RS in Gemeindeschlüssel umbenennen
regionalebenen['Gemeinde'] = regionalebenen['Gemeinde'].rename(columns={'_RS': 'Gemeindeschlüssel'})


In [301]:
# Überprüfen der ersten Zeilen der Tabellen

print("Gemeindetabelle:")
print(regionalebenen['Gemeinde'].head())

print("\nGemeindeverbandstabelle:")
print(regionalebenen['Gemeindeverband'].head())

print("\nKreis:")
print(regionalebenen['Stadtkreis_kreisfreie Stadt_Landkreis'].head())

print("\nLändertabelle:")
print(regionalebenen['Land'].head())

Gemeindetabelle:
   Gemeindeschlüssel                    Name  Gemeindeverbandsschlüssel
0        10010000000        Flensburg, Stadt                   10010000
1        10020000000  Kiel, Landeshauptstadt                   10020000
2        10030000000      Lübeck, Hansestadt                   10030000
3        10040000000       Neumünster, Stadt                   10040000
4        10510011011      Brunsbüttel, Stadt                   10510011

Gemeindeverbandstabelle:
                     Name  Gemeindeverbandsschlüssel  Kreisschlüssel
0  Burg-St. Michaelisdonn                   10515163            1051
1           Marne-Nordsee                   10515166            1051
2                   Eider                   10515169            1051
3           Heider Umland                   10515172            1051
4      Mitteldithmarschen                   10515175            1051

Kreis:


KeyError: 'Stadtkreis_kreisfreie Stadt_Landkreis'

## restliche tabellen 

In [ ]:
# Liste erstellen fuer die uebrigen dfs
tabellen_liste = [
    'CSV_Klassenstufe',
    'CSV_Schulform',
    'CSV_Hoechster_Schulabschluss',
    'CSV_Hoechster_berufl_Abschluss',
    'CSV_Erwerbsstatus',
    'CSV_ET_Alter',
    'CSV_ET_Hoechst_berufl_Abschl',
    'CSV_ET_Stellung_im_Beruf',
    'CSV_ET_Beruf_Hauptgr_ISCO08',
    'CSV_ET_Wirtschaftszweig'
]

In [ ]:
# Erstellen der Tabellen für alle definierten Tabellen und Konvertieren der Spalten

# Liste für die DataFrames
andere_tabellen = {}
# Iteration über die Liste der Tabellen
for tabelle in tabellen_liste:
    df = created_dfs[tabelle].drop(columns=['Berichtszeitpunkt', 'Name', 'Regionalebene']).drop_duplicates().reset_index(drop=True)
    df['Schluessel'] = df['_RS'].astype(int)# Erstellen einer Schluessel-Spalte aus der '_RS'-Spalte zu Integer
    df = df.drop(columns=['_RS'])
    df = df.fillna(0)  # NA-Werte durch 0 ersetzen
    # Konvertieren der Spalten in numerische Typen
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].apply(lambda x: int(x) if str(x).isdigit() else 0)
        else:
            df[col] = df[col].astype(int)
    # Verschieben der Schluessel-Spalte an die erste Position
    cols = ['Schluessel'] + [col for col in df if col != 'Schluessel']
    df = df[cols]
    andere_tabellen[tabelle] = df
    # check
    print(f"Tabelle {tabelle} wurde erfolgreich erstellt.")
    print(andere_tabellen[tabelle].head())


Tabelle CSV_Klassenstufe wurde erfolgreich erstellt.
    Schluessel  SCHUELER_KLSS_STP  SCHUELER_KLSS_STP__M  SCHUELER_KLSS_STP__W  \
0            0            8903780               4536040               4367740   
1            1             320060                162880                157180   
2         1001              10380                  5010                  5370   
3  10010000000              10380                  5010                  5370   
4         1002              24340                 12270                 12070   

   SCHUELER_KLSS_STP__1  SCHUELER_KLSS_STP__1_M  SCHUELER_KLSS_STP__1_W  \
0               3048570                 1563180                 1485400   
1                109240                   54930                   54310   
2                  3530                    1540                    1990   
3                  3530                    1540                    1990   
4                  8770                    4180                    4590   

   SCHUEL

# export

In [ ]:
# Verbindung zur SQLite-Datenbank herstellen
db_path = r'C:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\temp\zensus.db.session.sql'
if not os.path.exists(os.path.dirname(db_path)):
    raise FileNotFoundError(f"Der Pfad {os.path.dirname(db_path)} existiert nicht.")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [302]:
# Alle vorhandenen Tabellen droppen
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
for table in tables:
    cursor.execute(f'DROP TABLE IF EXISTS "{table[0]}";')

In [303]:
# Regionalebenen-Tabellen in die SQLite-Datenbank übertragen
for ebene, df in regionalebenen.items():
    df.to_sql(f'dim_{ebene.lower()}', conn, if_exists='replace', index=False)

In [304]:
# Andere Tabellen in die SQLite-Datenbank übertragen
for tabelle, df in andere_tabellen.items():
    df.to_sql(f'dim_{tabelle.lower()}', conn, if_exists='replace', index=False)

In [305]:
conn.close()
print("DataFrames erfolgreich in die SQLite-Datenbank übertragen.")

DataFrames erfolgreich in die SQLite-Datenbank übertragen.


# loeschen